In [0]:
%sql
CREATE OR REPLACE TEMP VIEW gen_seed AS
SELECT
  '@I_967018310@' AS person_id, --me
  0 AS generation_depth;

In [0]:
%sql
CREATE OR REPLACE TEMP VIEW person_parent_child AS
select person_id_1 as parent_id
, person_id_2 as child_id
from genealogy.silver_relationship
where relationship_type = 'parent'

In [0]:
%sql
CREATE OR REPLACE TABLE genealogy.gold_generation_depth AS
WITH RECURSIVE gen_all_steps(person_id, generation_depth, path) MAX RECURSION LEVEL 200 AS (
  -- Initial step: start from your initial gen_seed
  SELECT
    person_id,
    generation_depth,
    array(person_id) AS path
  FROM gen_seed

  UNION ALL

  -- Recursive step: repeat logic
  SELECT
    pc.parent_id AS person_id,
    g.generation_depth + 1 AS generation_depth,
    array_append(g.path, pc.parent_id)
  FROM gen_all_steps g
  JOIN person_parent_child pc
    ON g.person_id = pc.child_id
  WHERE pc.parent_id IS NOT NULL
)
SELECT
  person_id,
  MIN(generation_depth) AS generation_depth,
  MIN_BY(path, size(path)) AS path
FROM gen_all_steps
GROUP BY person_id;

In [0]:
%sql
CREATE OR REPLACE TEMP VIEW prox_seed AS
SELECT
  person_id,
  0 AS ancestral_proximity
FROM genealogy.gold_generation_depth;

In [0]:
%sql
CREATE OR REPLACE TABLE genealogy.gold_ancestral_proximity AS
WITH RECURSIVE prox_all_steps (
    person_id,
    ancestral_proximity,
    path,
    direct_ancestor_id
) AS (

  -- Seed
  SELECT
    person_id,
    0 AS ancestral_proximity,
    array(person_id) AS path,
    person_id AS direct_ancestor_id
  FROM prox_seed

  UNION ALL

  -- Recurse: parent->child only (adding spouses breaks things)
  SELECT
    r.next_person_id AS person_id,
    p.ancestral_proximity + 1,
    array_append(p.path, r.next_person_id),
    p.direct_ancestor_id
  FROM prox_all_steps p
  JOIN (

      -- parent -> child (existing direction)
      SELECT
        person_id_1 AS from_person_id,
        person_id_2 AS next_person_id
      FROM genealogy.silver_relationship
      WHERE relationship_type = 'parent'

/*
      UNION ALL

      -- spouse: husband -> wife
      SELECT
        person_id_1 AS from_person_id,
        person_id_2 AS next_person_id
      FROM genealogy.silver_relationship
      WHERE relationship_type = 'spouse'

      UNION ALL

      -- spouse: wife -> husband (reverse edge)
      SELECT
        person_id_2 AS from_person_id,
        person_id_1 AS next_person_id
      FROM genealogy.silver_relationship
      WHERE relationship_type = 'spouse'
*/
  ) r
    ON p.person_id = r.from_person_id

  WHERE
    -- cycle guard (mandatory with bidirectional edges)
    NOT array_contains(p.path, r.next_person_id)
)

SELECT
  person_id,
  MIN(ancestral_proximity) AS ancestral_proximity,
  MIN_BY(path, size(path)) AS path_to_ancestor,
  MIN_BY(direct_ancestor_id, size(path)) AS nearest_direct_ancestor_id
FROM prox_all_steps
GROUP BY person_id;


In [0]:
%sql
INSERT INTO genealogy.gold_ancestral_proximity
SELECT
  s.person_id_2 AS person_id, --wives
  p.ancestral_proximity + 1 AS ancestral_proximity,
  array_append(p.path_to_ancestor, s.person_id_2),
  p.nearest_direct_ancestor_id
FROM genealogy.gold_ancestral_proximity p
JOIN genealogy.silver_relationship s
  ON p.person_id = s.person_id_1
WHERE s.relationship_type = 'spouse'
AND NOT EXISTS (
  SELECT '1'
  FROM genealogy.gold_ancestral_proximity p2
  WHERE p2.person_id = s.person_id_2
);

INSERT INTO genealogy.gold_ancestral_proximity
SELECT
  s.person_id_1 AS person_id, --husbands
  p.ancestral_proximity + 1 AS ancestral_proximity,
  array_append(p.path_to_ancestor, s.person_id_1),
  p.nearest_direct_ancestor_id
FROM genealogy.gold_ancestral_proximity p
JOIN genealogy.silver_relationship s
  ON p.person_id = s.person_id_2
WHERE s.relationship_type = 'spouse'
AND NOT EXISTS (
  SELECT '1'
  FROM genealogy.gold_ancestral_proximity p2
  WHERE p2.person_id = s.person_id_1
);